# Lab 6 - Problemy wieloagentowe

**Wykonali:** Eryk Mikołajek, Jakub Kubicki, Marcin Zub

In [1]:
autorom_path = '/Users/marcin/.pyenv/versions/3.10.13/lib/python3.10/site-packages/AutoROM/roms'

In [9]:
import os
from pathlib import Path

import supersuit as ss
from pettingzoo.atari import pong_v3
from stable_baselines3 import PPO
from stable_baselines3.common.env_util import DummyVecEnv
from stable_baselines3.common.policies import ActorCriticPolicy
from stable_baselines3.common.callbacks import BaseCallback
from stable_baselines3.common.vec_env import VecEnvWrapper
from pettingzoo.utils.conversions import parallel_wrapper_fn
from gym import spaces
import numpy as np

In [12]:
!pip list

Package                      Version
---------------------------- -----------
absl-py                      2.1.0
ale-py                       0.8.1
appdirs                      1.4.4
appnope                      0.1.4
asttokens                    2.4.1
astunparse                   1.6.3
AutoROM                      0.6.1
AutoROM.accept-rom-license   0.6.1
box2d-py                     2.3.8
certifi                      2024.2.2
cffi                         1.16.0
charset-normalizer           3.3.2
cleanrl                      1.2.0
click                        8.1.7
cloudpickle                  3.0.0
comm                         0.2.2
contourpy                    1.2.1
cycler                       0.12.1
Cython                       3.0.10
debugpy                      1.8.1
decorator                    4.4.2
docker-pycreds               0.4.0
easyAI                       2.0.12
etils                        1.7.0
exceptiongroup               1.2.1
executing                    2.0.1
Faram

### Środowisko i proces uczenia

In [11]:
import gymnasium as gym

if autorom_path == '':
    print('Podaj ścieżkę do pliku autorom')
    exit(1)


env = gym.make('Pong-v4', max_episode_steps=100, render_mode='human')
print("Original observation space:", env.observation_space)

# Apply wrappers one by one, checking the observation space each time
env = ss.max_observation_v0(env, 2)
print("After max_observation_v0:", env.observation_space)

env = ss.sticky_actions_v0(env, repeat_action_probability=0.25)
print("After sticky_actions_v0:", env.observation_space)

env = ss.frame_skip_v0(env, 4)
print("After frame_skip_v0:", env.observation_space)

env = ss.frame_stack_v1(env, 4)
print("After frame_stack_v1:", env.observation_space)

# env.observation_space = gym.spaces.box.Box(0, 255, (210, 160, 3), np.uint8)
# env.action_space = gym.spaces.discrete.Discrete(6)

#### Callback do zapisu nagród
model = PPO("CnnPolicy", env, verbose=1)
# model.learn(total_timesteps=10000, callback=callback)

Original observation space: Box(0, 255, (210, 160, 3), uint8)
After max_observation_v0: Box(0, 255, (210, 160, 3), uint8)
After sticky_actions_v0: Box(0, 255, (210, 160, 3), uint8)
After frame_skip_v0: Box(0, 255, (210, 160, 3), uint8)
After frame_stack_v1: Box(0, 255, (210, 160, 12), uint8)
Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


AssertionError: The algorithm only supports (<class 'gym.spaces.box.Box'>, <class 'gym.spaces.discrete.Discrete'>, <class 'gym.spaces.multi_discrete.MultiDiscrete'>, <class 'gym.spaces.multi_binary.MultiBinary'>) as action spaces but Discrete(6) was provided

### Testowanie modelu

In [ ]:
obs = env.reset()
for _ in range(1000):
    action, _ = model.predict(obs)
    obs, rewards, dones, info = env.step(action)
    env.render()

env.close()

### Wykresy i krzywa uczenia

In [ ]:
import matplotlib.pyplot as plt

# Uśrednienie nagród na odcinkach
mean_rewards = [np.mean(callback.episode_rewards[i:i + 100]) for i in range(0, len(callback.episode_rewards), 100)]

plt.plot(mean_rewards)
plt.xlabel('Episodes (x100)')
plt.ylabel('Mean Total Reward')
plt.title('Learning Curve')
plt.show()